# ☕ BrewMaster: Cafe Management System
This notebook provides a comprehensive cafe management solution, including menu administration, order processing, and sales analytics.

### Features:
- **Backend**: SQLite database integration.
- **Menu Management**: Add and view menu items.
- **POS System**: Interactive ordering interface.
- **Analytics**: Visual sales summaries using Plotly.

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

class CafeBackend:
    def __init__(self, db_name="cafe.db"):
        self.db_name = db_name
        self.init_db()

    def init_db(self):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''CREATE TABLE IF NOT EXISTS menu (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name TEXT NOT NULL,
                                price REAL NOT NULL,
                                category TEXT
                            )''')
            cursor.execute('''CREATE TABLE IF NOT EXISTS orders (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                total REAL DEFAULT 0,
                                timestamp TEXT,
                                status TEXT DEFAULT 'Completed'
                            )''')
            cursor.execute('''CREATE TABLE IF NOT EXISTS order_items (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                order_id INTEGER,
                                menu_id INTEGER,
                                quantity INTEGER,
                                subtotal REAL,
                                FOREIGN KEY(order_id) REFERENCES orders(id),
                                FOREIGN KEY(menu_id) REFERENCES menu(id)
                            )''')
            
            cursor.execute("SELECT COUNT(*) FROM menu")
            if cursor.fetchone()[0] == 0:
                items = [
                    ('Espresso', 2.50, 'Coffee'),
                    ('Cappuccino', 3.50, 'Coffee'),
                    ('Latte', 3.75, 'Coffee'),
                    ('Blueberry Muffin', 2.95, 'Bakery'),
                    ('Croissant', 2.50, 'Bakery'),
                    ('Green Tea', 2.25, 'Tea')
                ]
                cursor.executemany("INSERT INTO menu (name, price, category) VALUES (?, ?, ?)", items)
            conn.commit()

    def get_menu(self):
        with sqlite3.connect(self.db_name) as conn:
            return pd.read_sql_query("SELECT * FROM menu", conn)

    def add_menu_item(self, name, price, category):
        with sqlite3.connect(self.db_name) as conn:
            conn.execute("INSERT INTO menu (name, price, category) VALUES (?, ?, ?)", (name, price, category))
            return True

    def create_order(self, items_dict):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO orders (timestamp) VALUES (?)", (datetime.now().strftime("%Y-%m-%d %H:%M:%S"),))
            order_id = cursor.lastrowid
            
            total = 0
            for mid, qty in items_dict.items():
                if qty <= 0: continue
                cursor.execute("SELECT price FROM menu WHERE id = ?", (mid,))
                price = cursor.fetchone()[0]
                subtotal = price * qty
                total += subtotal
                cursor.execute("INSERT INTO order_items (order_id, menu_id, quantity, subtotal) VALUES (?, ?, ?, ?)",
                               (order_id, mid, qty, subtotal))
            
            cursor.execute("UPDATE orders SET total = ? WHERE id = ?", (total, order_id))
            conn.commit()
            return order_id, total

    def get_orders(self):
        with sqlite3.connect(self.db_name) as conn:
            return pd.read_sql_query("SELECT * FROM orders ORDER BY id DESC", conn)

    def get_analytics(self):
        with sqlite3.connect(self.db_name) as conn:
            query = '''SELECT m.category, SUM(i.subtotal) as revenue 
                       FROM order_items i 
                       JOIN menu m ON i.menu_id = m.id 
                       GROUP BY m.category'''
            return pd.read_sql_query(query, conn)

cafe = CafeBackend()

## 📋 Menu Administration

In [ ]:
print("Current Menu:")
display(cafe.get_menu())

## 🛒 Interactive POS Terminal

In [ ]:
menu_df = cafe.get_menu()
inputs = {}
print("Place your order:")
for idx, row in menu_df.iterrows():
    inputs[row['id']] = widgets.IntText(description=f"{row['name']} (${row['price']})", value=0)
    display(inputs[row['id']])

btn = widgets.Button(description="Complete Order", button_style='success')
out = widgets.Output()
display(btn, out)

def handle_order(b):
    with out:
        clear_output()
        cart = {mid: inp.value for mid, inp.value in inputs.items()}
        if sum(cart.values()) == 0:
            print("Error: Cart is empty!")
            return
        oid, total = cafe.create_order(cart)
        print(f"✅ Success! Order #{oid} placed.")
        print(f"Total Bill: ${total:.2f}")

btn.on_click(handle_order)

## 📊 Sales Analytics

In [ ]:
orders = cafe.get_orders()
if not orders.empty:
    print(f"Total Revenue: ${orders['total'].sum():.2f}")
    print(f"Total Orders: {len(orders)}")
    
    analytics = cafe.get_analytics()
    fig = px.pie(analytics, values='revenue', names='category', title='Revenue by Category', hole=0.4)
    fig.show()
else:
    print("No sales data yet.")